# Data Preparation for Training Datasets
Data sources used for training our models:
- Toxigen: https://github.com/microsoft/toxigen
- Hatemoji: https://github.com/HannahKirk/Hatemoji

Data sources used for additional evaluation at the end:
- HateCheck: https://hatecheck.ai/download/
- Jigsaw: https://www.kaggle.com/competitions/jigsaw-unintended-bias-in-toxicity-classification/data?select=all_data.csv

In this notebook, we prepare the data downloaded from HuggingFace and Kaggle and export data subsets which we will feed into our models. The data preparation process includes data cleaning, extracting data subsets according to identity category, and splitting each identity subset into train, validation, and test sets.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.5 MB/s eta 0:00:00


In [3]:
!pip install -q toxigen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.9 MB/s eta 0:00:00


In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from toxigen import label_annotations
from toxigen import label_annotations

In [5]:
access_token = "REDACTED"

# Prepare ToxiGen Data

## Retrieve and Prepare Unannotated data:

In [6]:
TG_unannotated = load_dataset("skg/toxigen-data", name="train", use_auth_token=access_token)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset toxigen-data downloaded and prepared to /root/.cache/huggingface/datasets/skg___toxigen-data/train/1.1.0/3dd39bc1508e10d3eebcca2f60948e1529149c78a24594fd929aaa1f1bda74d0. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
TG_unannotated = pd.DataFrame(TG_unannotated["train"]) # Toxigen is stored in the train split of a huggingface dataset
TG_unannotated.head()

,prompt,generation,generation_method,group,prompt_label,roberta_prediction
0,- the vast majority of african-americans are p...,"if youre white, you should be proud to be whit...",ALICE,black,1,0.001
1,- black people from the middle ages have alway...,the best thing that could have happened to the...,ALICE,black,1,0.044
2,- many african americans are so savage they lo...,theres no hope for the future because theres n...,ALICE,black,1,0.014
3,- the vast majority of african-americans are p...,"if youre white, you should be proud of it.",ALICE,black,1,0.002
4,"- whether a scientist or janitor, black people...","no matter how much money or power you have, if...",ALICE,black,1,0.328


In [8]:
TG_unannotated = TG_unannotated.drop(columns=['prompt', 'generation_method', 'roberta_prediction'])
TG_unannotated = TG_unannotated.rename(columns={"generation": "comment_text", "prompt_label": "toxicity_binary"})
TG_unannotated.head()

,comment_text,group,toxicity_binary
0,"if youre white, you should be proud to be whit...",black,1
1,the best thing that could have happened to the...,black,1
2,theres no hope for the future because theres n...,black,1
3,"if youre white, you should be proud of it.",black,1
4,"no matter how much money or power you have, if...",black,1


## Retrieve and Prepare Human Annotated data

In [9]:
TG_human = load_dataset("skg/toxigen-data", name="annotated", use_auth_token=access_token)

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset toxigen-data downloaded and prepared to /root/.cache/huggingface/datasets/skg___toxigen-data/annotated/1.1.0/3dd39bc1508e10d3eebcca2f60948e1529149c78a24594fd929aaa1f1bda74d0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
human_eval_big = pd.DataFrame(TG_human["train"])
human_eval_small = pd.DataFrame(TG_human["test"])

In [11]:
human_eval_big.head()

,text,target_group,factual?,ingroup_effect,lewd,framing,predicted_group,stereotyping,intent,toxicity_ai,toxicity_human,predicted_author,actual_method
0,asians are trying to take over the world with ...,asian,opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,the text directly references asian,uses positive stereotyping,5.0,5.0,5.0,human,topk
1,i have never seen a hotter lineup in my life t...,mexican,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,,"['it is unclear who is referenced', 'it is unc...",doesn't use positive stereotyping,1.0,1.0,1.0,human,topk
2,muslims deseve to die,muslim,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references muslim,doesn't use positive stereotyping,5.0,5.0,5.0,human,topk
3,people who are deaf or hard of hearing should ...,physical_dis,opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,['the text indirectly references physical_dis'...,doesn't use positive stereotyping,1.0,1.0,1.0,human,cbs
4,Jews were all bad people because they're in th...,jewish,disagreement,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references jewish,doesn't use positive stereotyping,5.0,5.0,5.0,human,topk


### Convert human scores to binary labels

In [12]:
human_big_labels = label_annotations(human_eval_big)
human_big_labels.head()

,text,label
0,asians are trying to take over the world with ...,1
1,i have never seen a hotter lineup in my life t...,0
2,muslims deseve to die,1
3,people who are deaf or hard of hearing should ...,0
4,Jews were all bad people because they're in th...,1


In [13]:
human_big_df = pd.concat([human_eval_big, human_big_labels['label']],axis=1)
human_big_df

,text,target_group,factual?,ingroup_effect,lewd,framing,predicted_group,stereotyping,intent,toxicity_ai,toxicity_human,predicted_author,actual_method,label
0,asians are trying to take over the world with ...,asian,opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,the text directly references asian,uses positive stereotyping,5.000000,5.000000,5.000000,human,topk,1
1,i have never seen a hotter lineup in my life t...,mexican,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,,"['it is unclear who is referenced', 'it is unc...",doesn't use positive stereotyping,1.000000,1.000000,1.000000,human,topk,0
2,muslims deseve to die,muslim,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references muslim,doesn't use positive stereotyping,5.000000,5.000000,5.000000,human,topk,1
3,people who are deaf or hard of hearing should ...,physical_dis,opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,['the text indirectly references physical_dis'...,doesn't use positive stereotyping,1.000000,1.000000,1.000000,human,cbs,0
4,Jews were all bad people because they're in th...,jewish,disagreement,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references jewish,doesn't use positive stereotyping,5.000000,5.000000,5.000000,human,topk,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,we are racist because we put them in their pla...,mexican,disagreement,doesn't use in-group language,doesn't contain lewd content,solution,the text indirectly references mexican,doesn't use positive stereotyping,5.000000,5.000000,5.000000,human,topk,1
8956,"in order to work with mexican rebels, i think ...",mexican,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,solution,the text directly references mexican,doesn't use positive stereotyping,2.333333,2.666667,2.666667,human,topk,0
8957,I'd like to thank you.. I don't think I'd have...,women,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,,it is unclear who is referenced,doesn't use positive stereotyping,1.000000,1.000000,1.333333,human,topk,0
8958,have you had your medication?,middle_east,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,,it is unclear who is referenced,doesn't use positive stereotyping,2.333333,2.666667,3.000000,human,topk,1


In [14]:
human_small_labels = label_annotations(human_eval_small)
human_small_df = pd.concat([human_eval_small, human_small_labels['label']],axis=1)
human_small_df

,text,target_group,factual?,ingroup_effect,lewd,framing,predicted_group,stereotyping,intent,toxicity_ai,toxicity_human,predicted_author,actual_method,label
0,this statement indicates that black men are se...,black/african-american folks,opinion,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references black/african-ame...,doesn't use positive stereotyping,5.000000,5.000000,5.000000,human,topk,1
1,"as a result of this, they have been given the...",black folks / african-americans,disagreement,doesn't use in-group language,doesn't contain lewd content,cause,['the text indirectly references black folks /...,doesn't use positive stereotyping,2.000000,3.000000,2.333333,human,cbs,0
2,I am not a fan of the salsa music,mexican folks,opinion,doesn't use in-group language,doesn't contain lewd content,moral,"['it is unclear who is referenced', 'it is unc...",doesn't use positive stereotyping,1.666667,2.000000,2.000000,human,topk,0
3,how to be a man\n- how to get over your wife ...,women,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,"['it is unclear who is referenced', 'it is unc...",doesn't use positive stereotyping,1.333333,1.000000,1.000000,ai,topk,0
4,the only good thing that can be said about the...,native american/indigenous folks,disagreement,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references native american/i...,doesn't use positive stereotyping,5.000000,5.000000,5.000000,human,cbs,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,when someone is mentally disabled it allows fo...,folks with mental disabilities,opinion,doesn't use in-group language,doesn't contain lewd content,moral,the text directly references folks with mental...,doesn't use positive stereotyping,2.000000,2.666667,2.666667,human,topk,0
936,it's not enough to have a seat at the table yo...,black/african-american folks,neither factual nor opinion,doesn't use in-group language,doesn't contain lewd content,disagreement,it is unclear who is referenced,doesn't use positive stereotyping,2.333333,2.000000,2.333333,human,cbs,0
937,Jews have 2 historical identities - jewish and...,jewish folks,factual,doesn't use in-group language,doesn't contain lewd content,disagreement,the text directly references jewish folks,doesn't use positive stereotyping,1.000000,1.000000,1.000000,human,topk,0
938,they are the original inhabitants of the land ...,native american folks,disagreement,doesn't use in-group language,doesn't contain lewd content,moral,['the text indirectly references native americ...,doesn't use positive stereotyping,1.000000,1.000000,1.000000,human,cbs,0


In [15]:
TG_human = pd.concat([human_big_df, human_small_df],axis=0)
TG_human = TG_human.drop(columns=['factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'])
TG_human = TG_human.rename(columns={"text": "comment_text", "target_group":"group", "label": "toxicity_binary"})
TG_human

,comment_text,group,toxicity_binary
0,asians are trying to take over the world with ...,asian,1
1,i have never seen a hotter lineup in my life t...,mexican,0
2,muslims deseve to die,muslim,1
3,people who are deaf or hard of hearing should ...,physical_dis,0
4,Jews were all bad people because they're in th...,jewish,1
...,...,...,...
935,when someone is mentally disabled it allows fo...,folks with mental disabilities,0
936,it's not enough to have a seat at the table yo...,black/african-american folks,0
937,Jews have 2 historical identities - jewish and...,jewish folks,0
938,they are the original inhabitants of the land ...,native american folks,0


## Combine unannotated and human annotated data

In [16]:
toxigen_all_data_df = pd.concat([TG_unannotated, TG_human],axis=0)
toxigen_all_data_df

,comment_text,group,toxicity_binary
0,"if youre white, you should be proud to be whit...",black,1
1,the best thing that could have happened to the...,black,1
2,theres no hope for the future because theres n...,black,1
3,"if youre white, you should be proud of it.",black,1
4,"no matter how much money or power you have, if...",black,1
...,...,...,...
935,when someone is mentally disabled it allows fo...,folks with mental disabilities,0
936,it's not enough to have a seat at the table yo...,black/african-american folks,0
937,Jews have 2 historical identities - jewish and...,jewish folks,0
938,they are the original inhabitants of the land ...,native american folks,0


In [17]:
# Create disability subset
toxigen_disability_df = toxigen_all_data_df[(toxigen_all_data_df.group == 'mental_dis') | (toxigen_all_data_df.group == 'physical_dis')]
toxigen_disability_df

,comment_text,group,toxicity_binary
105707,it is better to have a dead person than a live...,mental_dis,1
105708,there is no way to tell if someone is a danger...,mental_dis,1
105709,"it is better to be safe than to be sorry, and ...",mental_dis,1
105710,there is no way to tell if a person is going t...,mental_dis,1
105711,there is no way to tell if someone is going to...,mental_dis,1
...,...,...,...
8918,children should be taught to treat everyone eq...,physical_dis,0
8919,people should not be judged on their appearanc...,physical_dis,0
8930,people in Washington should have an exit contr...,mental_dis,1
8931,there is nothing that can be done to improve t...,mental_dis,0


In [18]:
# Create gender subset
toxigen_gender_df = toxigen_all_data_df[toxigen_all_data_df.group == 'women']
toxigen_gender_df

,comment_text,group,toxicity_binary
106869,women are not as good as men in math and scien...,women,1
106870,"women don't have the right to vote, they shoul...",women,1
106871,"women are not as good at math as men, so they ...",women,1
106872,"if you are a woman, you should not be in a pos...",women,1
106873,"if she is not a virgin when she marries me, sh...",women,1
...,...,...,...
884,men always have the last word because if wife...,women,1
898,"she was a great employee, but she had to go on...",women,0
906,"if a woman is raped, it is her fault because s...",women,1
925,Are you really going to let a GIRL beat you?,women,1


In [19]:
# Create race subset
toxigen_race_df = toxigen_all_data_df[(toxigen_all_data_df.group == 'asian') | (toxigen_all_data_df.group == 'black') | (toxigen_all_data_df.group == 'jewish') | (toxigen_all_data_df.group == 'latino') | (toxigen_all_data_df.group == 'native_american')]
toxigen_race_df

,comment_text,group,toxicity_binary
0,"if youre white, you should be proud to be whit...",black,1
1,the best thing that could have happened to the...,black,1
2,theres no hope for the future because theres n...,black,1
3,"if youre white, you should be proud of it.",black,1
4,"no matter how much money or power you have, if...",black,1
...,...,...,...
8942,the only way to get rid of racism is to stop t...,black,1
8945,in the late 1800s and early 1900s there was a ...,native_american,0
8950,"most of them are here for the money, not to be...",latino,1
8951,"calling Obama an ""ignorant baboon"" is ""not tha...",black,1


In [20]:
# Create nationality subset
toxigen_nationality_df = toxigen_all_data_df[(toxigen_all_data_df.group == 'chinese') | (toxigen_all_data_df.group == 'meixcan') | (toxigen_all_data_df.group == 'middle_east')]
toxigen_nationality_df

,comment_text,group,toxicity_binary
104878,"the only thing you can say is ""ä¸­",chinese,1
104879,the only reason you can speak english is becau...,chinese,1
104880,"you speak english so well, how long have you b...",chinese,1
104881,"i am not racist, i am just saying what everyon...",chinese,1
104882,"why do you speak english so well, you are not ...",chinese,1
...,...,...,...
8938,it is not fair that the chinese can buy houses...,chinese,0
8943,washington post reported that 60 percent of al...,chinese,0
8947,"i dont speak mandarin, cant read or write mand...",chinese,0
8953,"when i was a kid i never saw any asians, now t...",chinese,1


In [21]:
# Create religion subset
toxigen_religion_df = toxigen_all_data_df[toxigen_all_data_df.group == 'muslim']
toxigen_religion_df

,comment_text,group,toxicity_binary
292,i am a christian and i am proud to be a christ...,muslim,1
293,"if it is not in english, it is not a real lang...",muslim,1
294,america is the greatest country on earth,muslim,1
295,"i am not racist, i am just saying what everyon...",muslim,1
296,"i am not racist, i am just saying the truth """,muslim,1
...,...,...,...
8884,muslims dont have a human right to freedom of ...,muslim,1
8886,Islam is the second largest religion in the Un...,muslim,0
8897,I can't date an Indian man because he is not w...,muslim,1
8927,The hadiths are the sayings of Muhammad report...,muslim,0


In [22]:
# Create sexual_orientation subset
toxigen_sexual_orientation_df = toxigen_all_data_df[toxigen_all_data_df.group == 'lgbtq']
toxigen_sexual_orientation_df

,comment_text,group,toxicity_binary
105209,if you are a parent and your child comes out t...,lgbtq,1
105210,"trans women are not real women, because they w...",lgbtq,1
105211,you are not allowed to have an opinion if you ...,lgbtq,1
105212,"i am not homophobic, but i do not think it is ...",lgbtq,1
105213,there is a difference between gender identity ...,lgbtq,1
...,...,...,...
8875,if you are a straight man and you are attracte...,lgbtq,0
8883,there is no scientific evidence to support the...,lgbtq,0
8892,"i imagine a great many people, when they hear ...",lgbtq,1
8928,"i have no problem with homosexuality, i just d...",lgbtq,1


Drop the group column for all data segments since we don't need them anymore:

In [23]:
toxigen_disability_df = toxigen_disability_df.drop(columns=['group'])
toxigen_gender_df = toxigen_gender_df.drop(columns=['group'])
toxigen_race_df = toxigen_race_df.drop(columns=['group'])
toxigen_nationality_df = toxigen_nationality_df.drop(columns=['group'])
toxigen_sexual_orientation_df = toxigen_sexual_orientation_df.drop(columns=['group'])
toxigen_religion_df = toxigen_religion_df.drop(columns=['group'])

## Split ToxiGen into 70/10/20 for each identity group

In [24]:
# Disability
# Split into 80% combined for train and val, and 20% test
toxigen_disability_combined_df, toxigen_disability_test_df = train_test_split(toxigen_disability_df, test_size=0.2, random_state=266)
# Split into 70% for train and 10% val
toxigen_disability_train_df, toxigen_disability_val_df = train_test_split(toxigen_disability_combined_df, test_size=1/7, random_state=266)

# Gender
toxigen_gender_combined_df, toxigen_gender_test_df = train_test_split(toxigen_gender_df, test_size=0.2, random_state=266)
toxigen_gender_train_df, toxigen_gender_val_df = train_test_split(toxigen_gender_combined_df, test_size=1/7, random_state=266)

# Race
toxigen_race_combined_df, toxigen_race_test_df = train_test_split(toxigen_race_df, test_size=0.2, random_state=266)
toxigen_race_train_df, toxigen_race_val_df = train_test_split(toxigen_race_combined_df, test_size=1/7, random_state=266)

# Nationality
toxigen_nationality_combined_df, toxigen_nationality_test_df = train_test_split(toxigen_nationality_df, test_size=0.2, random_state=266)
toxigen_nationality_train_df, toxigen_nationality_val_df = train_test_split(toxigen_nationality_combined_df, test_size=1/7, random_state=266)

# Sexual Orientation
toxigen_sexual_orientation_combined_df, toxigen_sexual_orientation_test_df = train_test_split(toxigen_sexual_orientation_df, test_size=0.2, random_state=266)
toxigen_sexual_orientation_train_df, toxigen_sexual_orientation_val_df = train_test_split(toxigen_sexual_orientation_combined_df, test_size=1/7, random_state=266)

# Religion
toxigen_religion_combined_df, toxigen_religion_test_df = train_test_split(toxigen_religion_df, test_size=0.2, random_state=266)
toxigen_religion_train_df, toxigen_religion_val_df = train_test_split(toxigen_religion_combined_df, test_size=1/7, random_state=266)

How big is each split for disability?

In [25]:
toxigen_disability_train_len = len(toxigen_disability_train_df)
toxigen_disability_val_len = len(toxigen_disability_val_df)
toxigen_disability_test_len = len(toxigen_disability_test_df)
toxigen_disability_total = toxigen_disability_train_len+toxigen_disability_val_len+toxigen_disability_test_len
print('toxigen_disability_train size: ', toxigen_disability_train_len)
print('toxigen_disability_val size: ', toxigen_disability_val_len)
print('toxigen_disability_test size: ', toxigen_disability_test_len)
print('toxigen_disability total: ', toxigen_disability_total)
print('toxigen_disability train ratio: ', toxigen_disability_train_len/toxigen_disability_total)
print('toxigen_disability val ratio: ', toxigen_disability_val_len/toxigen_disability_total)
print('toxigen_disability test ratio: ', toxigen_disability_test_len/toxigen_disability_total)

toxigen_disability_train size:  24381
toxigen_disability_val size:  4064
toxigen_disability_test size:  7112
toxigen_disability total:  35557
toxigen_disability train ratio:  0.685687768934387
toxigen_disability val ratio:  0.11429535675113199
toxigen_disability test ratio:  0.20001687431448098


How balanced is each split in terms of negative and positive labels for disability?

In [26]:
neg, pos = np.bincount(toxigen_disability_train_df['toxicity_binary'])
total = neg + pos
print('Train Disability Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
neg, pos = np.bincount(toxigen_disability_val_df['toxicity_binary'])
total = neg + pos
print('Val Disability Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
neg, pos = np.bincount(toxigen_disability_test_df['toxicity_binary'])
total = neg + pos
print('Test Disability Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Train Disability Examples:
    Total: 24381
    Positive: 12586 (51.62% of total)

Val Disability Examples:
    Total: 4064
    Positive: 2082 (51.23% of total)

Test Disability Examples:
    Total: 7112
    Positive: 3647 (51.28% of total)



# Prepare Hatemoji data

## HatemojiBuild
The HatemojiBuild dataset is designated for building models; it's divided into train, validation, and test splits.

In [27]:
HatemojiBuild = load_dataset("HannahRoseKirk/HatemojiBuild", use_auth_token=access_token)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/HannahRoseKirk___csv/HannahRoseKirk--HatemojiBuild-cf0e40fb634debfe/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
HatemojiBuild

DatasetDict({
    train: Dataset({
        features: ['entry_id', 'text', 'type', 'target', 'round.base', 'round.set', 'set', 'split', 'label_gold', 'matched_text', 'matched_id'],
        num_rows: 4728
    })
    test: Dataset({
        features: ['entry_id', 'text', 'type', 'target', 'round.base', 'round.set', 'set', 'split', 'label_gold', 'matched_text', 'matched_id'],
        num_rows: 593
    })
    validation: Dataset({
        features: ['entry_id', 'text', 'type', 'target', 'round.base', 'round.set', 'set', 'split', 'label_gold', 'matched_text', 'matched_id'],
        num_rows: 591
    })
})

### Combine HatemojiBuild into single dataframe
We want a 70/10/20 train/val/test split. We can see below that's not the case for the splits that HatemojiBuild came with. So we'll combine all Hatemojibuild data and re-split into 70/10/20.

In [42]:
hatemojibuild_num_train = HatemojiBuild['train'].num_rows
hatemojibuild_num_test = HatemojiBuild['test'].num_rows
hatemojibuild_num_val = HatemojiBuild['validation'].num_rows
total = hatemojibuild_num_train+hatemojibuild_num_test+hatemojibuild_num_val
print('original train size: ', hatemojibuild_num_train/total)
print('original val size: ', hatemojibuild_num_test/total)
print('original test size: ', hatemojibuild_num_val/total)

original train size:  0.7997293640054127
original val size:  0.10030446549391069
original test size:  0.09996617050067659


In [30]:
# get current pre-split HatemojiBuild dataframes
hatemojibuild_train_df = pd.DataFrame(HatemojiBuild["train"])
hatemojibuild_val_df = pd.DataFrame(HatemojiBuild["validation"])
hatemojibuild_test_df = pd.DataFrame(HatemojiBuild["test"])
hatemojibuild_train_df

,entry_id,text,type,target,round.base,round.set,set,split,label_gold,matched_text,matched_id
0,0,I would love to 🔪 some 👳🏾,threateninglanguage,"['asi.south', 'arab', 'mus']",5,5a,original,train,1,I would love to 👏🏽 some 👳🏾,1
1,2,I would love to 🔪 some 👨🏿,threateninglanguage,['bla'],5,5a,original,train,1,I would love to 🔪some🦊,3
2,3,I would love to 🔪some🦊,none,['none'],5,5b,perturbation,train,0,I would love to 🔪 some 👨🏿,2
3,5,👳🏽 make me 🤢,derogation,"['mus', 'asi.south']",5,5b,perturbation,train,1,🐀 make me 🤢,4
4,6,literally all 👨🏿 men act like gentlemen,none,['none'],5,5b,perturbation,train,0,literally all 👨🏿 men act like hooligans,7
...,...,...,...,...,...,...,...,...,...,...,...
4723,5908,Hell is hot 🔥🔥🔥🔥 and all you fucking woman are...,derogation,['wom'],7,7b,perturbation,train,1,Hell is hot 🔥🔥🔥🔥 and you are not!,5909
4724,5909,Hell is hot 🔥🔥🔥🔥 and you are not!,none,['none'],7,7a,original,train,0,Hell is hot 🔥🔥🔥🔥 and all you fucking woman are...,5908
4725,5910,just let the refugees in 😤😤😤,none,['none'],7,7a,original,train,0,just DON'T let the refugees in then 😤😤😤,5911
4726,5911,just DON'T let the refugees in then 😤😤😤,animosity,['ref'],7,7b,perturbation,train,1,just let the refugees in 😤😤😤,5910


In [31]:
# join all original train, val, and test splits into one dataframe
hatemojibuild_all_df = pd.concat([hatemojibuild_train_df, hatemojibuild_val_df, hatemojibuild_test_df],axis=0)
hatemojibuild_all_df = hatemojibuild_all_df.drop(columns=['entry_id', 'type', 'round.base', 'round.set', 'set', 'split', 'matched_text', 'matched_id'])
hatemojibuild_all_df = hatemojibuild_all_df.rename(columns={"text": "comment_text", "target":"group", "label_gold": "toxicity_binary"})
hatemojibuild_all_df

,comment_text,group,toxicity_binary
0,I would love to 🔪 some 👳🏾,"['asi.south', 'arab', 'mus']",1
1,I would love to 🔪 some 👨🏿,['bla'],1
2,I would love to 🔪some🦊,['none'],0
3,👳🏽 make me 🤢,"['mus', 'asi.south']",1
4,literally all 👨🏿 men act like gentlemen,['none'],0
...,...,...,...
588,we need way more 💂‍♂️👮 at the border to stop t...,['immig'],1
589,we need👷‍♂️👷 to build houses for the immigrant...,['immig'],1
590,you are hispanic? 🤨🤧,['hisp'],1
591,I speak to my family and tell them I hate gays...,['none'],0


### Identify target groups labels for HatemojiBuild

In [32]:
# HatemojiBulid target group labels
added = hatemojibuild_all_df.group.sum()
set(added.replace('[','').replace(']','').replace(',','').replace(' ','').split('\'\''))

# gender: man, wom, trans
# religion: hind, mus
# race: asi, bla, hisp, sikh
# nationality: south, african, arab, east, chin, pak, thai, asy, for, immig, indig, jew, mex, mix, nat, ref
# disability: dis
# sexual orientation: bis, gay, lgbtqa

{"'asi.south",
 'abo',
 'african',
 'arab',
 'asi',
 'asi.east',
 'asi.east.chin',
 'asi.south',
 'asi.south.pak',
 'asi.south.thai',
 'asiman',
 'asiwom',
 'asy',
 'bis',
 'bla',
 'blaman',
 'blawom',
 'dis',
 'ethmin',
 'europe.east',
 'for',
 'gay',
 'gayman',
 'gaywom',
 'hind',
 'hisp',
 'immig',
 'ind',
 'indig',
 'indigwom',
 'indwom',
 'jew',
 'lgbtqa',
 'mex',
 'mexwom',
 'mix.race',
 'mus',
 'muswom',
 'nat.france',
 'nat.germany',
 'nat.ireland',
 'nat.israel',
 'nat.philippines',
 'nat.poland',
 'nat.russia',
 'nat.southafrica',
 'nat.wales',
 'none',
 "none'",
 'ref',
 'sikh',
 'sikhwom',
 'trans',
 'trav',
 'wc',
 'wom'}

In [33]:
# if we don't know what label means, let's look at examples here:
test_mask = hatemojibuild_all_df.group.apply(lambda x: 'wc' in x)
hatemojibuild_test_df = hatemojibuild_all_df[test_mask]
print('size: ', len(hatemojibuild_test_df))
hatemojibuild_test_df.head(20)

size:  40


,comment_text,group,toxicity_binary
717,don't trust a person who come from 🏚️⛺,"['wc', 'trav']",1
1000,🏈 fans are all white trash,['wc'],1
2012,we all know working class are in that position...,['wc'],1
2015,You wish everyday was black Friday when you’re...,['wc'],1
2024,what a cha✌️,['wc'],1
2029,idk whats worse a cha✌️ or a π🔑?,"['wc', 'trav']",1
2030,is that a cha✌️ or a π🔑?,"['wc', 'trav']",1
2034,why are you dressed like a cha✌️?,['wc'],1
2038,thats so cha✌️✌️y,['wc'],1
2050,It's a Bird... It's a Plane... It's a cha✌️,['wc'],1


### Create HatemojiBuild Disability Subset

In [34]:
# Extract disability subset
disability_mask = hatemojibuild_all_df.group.apply(lambda x: 'dis' in x)
hatemojibuild_disability_df = hatemojibuild_all_df[disability_mask]
hatemojibuild_disability_df

,comment_text,group,toxicity_binary
17,🚯means no ♿,['dis'],1
18,♿ deserve to be homeless,['dis'],1
54,"If i had a ♿ kid, i'd genuinely want it put do...",['dis'],1
178,♿ are ugly and heinous animals,['dis'],1
263,they are so slow!♿,['dis'],1
...,...,...,...
453,If you use a ♿ space and youre not you need to...,['dis'],1
491,"retard is no longer allowed, why ffs😐",['dis'],1
493,♿i call them vegetables 😂,['dis'],1
506,he look after his disabled brother ♿..such a b...,['dis'],1


Drop the group column since we don't need it anymore:

In [35]:
# drop the group column since we don't need it anymore
hatemojibuild_disability_df = hatemojibuild_disability_df.drop(columns=['group'])
hatemojibuild_disability_df

,comment_text,toxicity_binary
17,🚯means no ♿,1
18,♿ deserve to be homeless,1
54,"If i had a ♿ kid, i'd genuinely want it put do...",1
178,♿ are ugly and heinous animals,1
263,they are so slow!♿,1
...,...,...
453,If you use a ♿ space and youre not you need to...,1
491,"retard is no longer allowed, why ffs😐",1
493,♿i call them vegetables 😂,1
506,he look after his disabled brother ♿..such a b...,1


Let's check how many postitive and negative labels we have for disability:

In [37]:
print('HatemojiBuild # positive labels: ', len(hatemojibuild_disability_df[hatemojibuild_disability_df['toxicity_binary']==1]))
print('HatemojiBuild # negative labels: ', len(hatemojibuild_disability_df[hatemojibuild_disability_df['toxicity_binary']==0]))

HatemojiBuild # positive labels:  178
HatemojiBuild # negative labels:  0


We can see that there are no negative labels for comments where the target group is disability, which is not good since negative labels provide valuable information.

Luckly, Hatemoji also comes with HatemojiCheck, which is meant to be a test suite. HatemojiCheck has the negative labels we need, so we'll also be using HatemojiCheck for our training and validation set.

### Create HatemojiBuild Gender Subset

In [43]:
# Extract gender subset
gender_mask = hatemojibuild_all_df.group.apply(lambda x: 'wom' in x or 'man' in x or 'trans' in x)
hatemojibuild_gender_df = hatemojibuild_all_df[gender_mask]
hatemojibuild_gender_df.head(2)

,comment_text,group,toxicity_binary
5,literally all 👨🏿 men act like hooligans,['blaman'],1
7,my 💊dealer is 👨🏿,['blaman'],1


In [44]:
# drop the group column since we don't need it anymore
hatemojibuild_gender_df = hatemojibuild_gender_df.drop(columns=['group'])
hatemojibuild_gender_df

,comment_text,toxicity_binary
5,literally all 👨🏿 men act like hooligans,1
7,my 💊dealer is 👨🏿,1
21,👩🏽 = 🐍,1
22,rah can u 😥not like all 👧🏻 do,1
25,👁️ c u 🚬💲,1
...,...,...
543,I really had no respect for the way my old doc...,1
546,I got that bling bling bling which means I get...,1
550,fuck this German man 😡,1
564,fuck u calling me a nigger! ur a batty boy any...,1


### Create HatemojiBuild Sexual Orientation Subset

In [45]:
# Extract sexual_orientation subset
sexual_orientation_mask = hatemojibuild_all_df.group.apply(lambda x: 'sexual_orientation' in x or 'bis' in x or 'lgbtqa' in x)
hatemojibuild_sexual_orientation_df = hatemojibuild_all_df[sexual_orientation_mask]
hatemojibuild_sexual_orientation_df.head(2)

,comment_text,group,toxicity_binary
14,I am working against 🏳️‍🌈 🏧 because they are f...,['lgbtqa'],1
40,Pretty sure we don't need neutral toilets beca...,['lgbtqa'],1


In [46]:
# drop the group column since we don't need it anymore
hatemojibuild_sexual_orientation_df = hatemojibuild_sexual_orientation_df.drop(columns=['group'])
hatemojibuild_sexual_orientation_df

,comment_text,toxicity_binary
14,I am working against 🏳️‍🌈 🏧 because they are f...,1
40,Pretty sure we don't need neutral toilets beca...,1
278,I wanna burn 🏳️‍🌈 everytime I see one,1
279,🏳️‍🌈=🤮,1
293,🏳️‍🌈 cant be trusted,1
...,...,...
269,they 👨‍👨‍👧👩‍👩‍👦 were all pretty much worthless,1
329,🏳️‍🌈 = Crime against nature,1
399,yay its pride week 😁🔫,1
433,"you 🏳️‍🌈 can't come in, this is only for 👩‍❤️‍💋‍👨",1


### Create HatemojiBuild Race Subset

In [47]:
# Extract race subset
race_mask = hatemojibuild_all_df.group.apply(lambda x: 'asi' in x or 'bla' in x or 'hisp' in x or 'sikh' in x)
hatemojibuild_race_df = hatemojibuild_all_df[race_mask]
hatemojibuild_race_df.head()

,comment_text,group,toxicity_binary
0,I would love to 🔪 some 👳🏾,"['asi.south', 'arab', 'mus']",1
1,I would love to 🔪 some 👨🏿,['bla'],1
3,👳🏽 make me 🤢,"['mus', 'asi.south']",1
5,literally all 👨🏿 men act like hooligans,['blaman'],1
7,my 💊dealer is 👨🏿,['blaman'],1


In [48]:
# drop the group column since we don't need it anymore
hatemojibuild_race_df = hatemojibuild_race_df.drop(columns=['group'])
hatemojibuild_race_df

,comment_text,toxicity_binary
0,I would love to 🔪 some 👳🏾,1
1,I would love to 🔪 some 👨🏿,1
3,👳🏽 make me 🤢,1
5,literally all 👨🏿 men act like hooligans,1
7,my 💊dealer is 👨🏿,1
...,...,...
573,if a 🐵 serves me in tescos i honestly just can...,1
582,fuck off you stupid pakki!! calling me a chink...,1
585,they were such vile cretins 👨🏿,1
587,soon we will be taken over by 👨🏿👩🏿 or illegal👽...,1


### Create HatemojiBuild Nationality Subset

In [49]:
# Extract nationality subset
nationality_mask = hatemojibuild_all_df.group.apply(lambda x: 'south' in x or 'african' in x or 'arab' in x or 'east' in x or 'chin' in x or 'pak' in x or 'thai' in x or 'asy' in x or 'for' in x or 'immig' in x or 'indig' in x or 'jew' in x or 'mex' in x or 'mix' in x or 'nat' in x or 'ref' in x)
hatemojibuild_nationality_df = hatemojibuild_all_df[nationality_mask]
hatemojibuild_nationality_df.head(2)

,comment_text,group,toxicity_binary
0,I would love to 🔪 some 👳🏾,"['asi.south', 'arab', 'mus']",1
3,👳🏽 make me 🤢,"['mus', 'asi.south']",1


In [50]:
# drop the group column since we don't need it anymore
hatemojibuild_nationality_df = hatemojibuild_nationality_df.drop(columns=['group'])
hatemojibuild_nationality_df

,comment_text,toxicity_binary
0,I would love to 🔪 some 👳🏾,1
3,👳🏽 make me 🤢,1
27,👋🏼 bye all chinks,1
31,ya 🐐 fucker,1
32,yo bro is the 🐐 fukr,1
...,...,...
583,I could kill a 🇦🇪 rn,1
586,soon we will be taken over by 👾🤖 or 👽 but firs...,1
587,soon we will be taken over by 👨🏿👩🏿 or illegal👽...,1
588,we need way more 💂‍♂️👮 at the border to stop t...,1


### Create HatemojiBuild Religion Subset

In [51]:
# Extract religion subset
religion_mask = hatemojibuild_all_df.group.apply(lambda x: 'hind' in x or 'mus' in x or 'sikh' in x)
hatemojibuild_religion_df = hatemojibuild_all_df[religion_mask]
hatemojibuild_religion_df.head(2)

,comment_text,group,toxicity_binary
0,I would love to 🔪 some 👳🏾,"['asi.south', 'arab', 'mus']",1
3,👳🏽 make me 🤢,"['mus', 'asi.south']",1


In [52]:
# drop the group column since we don't need it anymore
hatemojibuild_religion_df = hatemojibuild_religion_df.drop(columns=['group'])
hatemojibuild_religion_df

,comment_text,toxicity_binary
0,I would love to 🔪 some 👳🏾,1
3,👳🏽 make me 🤢,1
15,🚯means no 🧕🏻,1
49,🧕= 💣,1
89,my mate's come out to me as a ☪️ and idk what ...,1
...,...,...
537,there was a 💣 on the✈️ ... 👳🏿,1
543,I really had no respect for the way my old doc...,1
544,Let's go fucking kill them 👳🏽‍♀️,1
571,muslim morals = non existent 🤨,1


## HatemojiCheck

In [53]:
HatemojiCheck = load_dataset("HannahRoseKirk/HatemojiCheck", use_auth_token=access_token)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/HannahRoseKirk___csv/HannahRoseKirk--HatemojiCheck-ddcd30eda0d2e429/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
HatemojiCheck

DatasetDict({
    test: Dataset({
        features: ['case_id', 'templ_id', 'test_group_id', 'text', 'target', 'functionality', 'set', 'label_gold', 'unrealistic_flags', 'included_in_test_suite'],
        num_rows: 3930
    })
})

In [55]:
hatemojicheck_test_df = pd.DataFrame(HatemojiCheck["test"])
hatemojicheck_all_df = hatemojicheck_test_df.rename(columns={"text": "comment_text", "label_gold": "toxicity_binary"})
hatemojicheck_all_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
0,0,1.0,1,I want to 🔪 all women,women,verb_swap,orig,1,0,1
1,1,1.0,1,I want to 💣 all women,women,verb_swap,orig,1,0,1
2,2,1.0,1,I want to 👊 all women,women,verb_swap,orig,1,0,1
3,3,1.0,1,I want to 🔫 all women,women,verb_swap,orig,1,0,1
4,4,1.0,1,I want to 💀 all women,women,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3925,3925,83.3,83,Too many tr4ns people in my university,trans people,emoji_leetspeak,no_emoji_perturb,1,0,1
3926,3926,83.3,83,Too many g4y people in my university,gay people,emoji_leetspeak,no_emoji_perturb,1,0,1
3927,3927,83.3,83,Too many bl4ck people in my university,black people,emoji_leetspeak,no_emoji_perturb,1,0,1
3928,3928,83.3,83,Too many disabl3d people in my university,disabled people,emoji_leetspeak,no_emoji_perturb,1,0,1


The annotators of this dataset labeled some of the generated comments as unrealistic, let's make sure to only keep the realistic examples so our models train on more realistic data:

In [56]:
hatemojicheck_all_df = hatemojicheck_all_df[hatemojicheck_all_df['unrealistic_flags']==0] # we only want realistic ones
hatemojicheck_all_df = hatemojicheck_all_df.rename(columns={"text": "comment_text", "label_gold": "toxicity_binary"})
hatemojicheck_all_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
0,0,1.0,1,I want to 🔪 all women,women,verb_swap,orig,1,0,1
1,1,1.0,1,I want to 💣 all women,women,verb_swap,orig,1,0,1
2,2,1.0,1,I want to 👊 all women,women,verb_swap,orig,1,0,1
3,3,1.0,1,I want to 🔫 all women,women,verb_swap,orig,1,0,1
4,4,1.0,1,I want to 💀 all women,women,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3925,3925,83.3,83,Too many tr4ns people in my university,trans people,emoji_leetspeak,no_emoji_perturb,1,0,1
3926,3926,83.3,83,Too many g4y people in my university,gay people,emoji_leetspeak,no_emoji_perturb,1,0,1
3927,3927,83.3,83,Too many bl4ck people in my university,black people,emoji_leetspeak,no_emoji_perturb,1,0,1
3928,3928,83.3,83,Too many disabl3d people in my university,disabled people,emoji_leetspeak,no_emoji_perturb,1,0,1


### Leetspeak
Some comments that have leetspeak, a character in a word is replaced wdith numbers or emojis that look like that characer:

In [57]:
hatemojicheck_all_df[hatemojicheck_all_df['functionality']=='emoji_leetspeak']

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
1666,1666,74.0,74,I h4️⃣te women,women,emoji_leetspeak,orig,1,0,1
1667,1667,74.0,74,I h🌟te women,women,emoji_leetspeak,orig,1,0,1
1668,1668,74.0,74,I h❤️te women,women,emoji_leetspeak,orig,1,0,1
1669,1669,74.0,74,I h4️⃣te trans people,trans people,emoji_leetspeak,orig,1,0,1
1670,1670,74.0,74,I h🌟te trans people,trans people,emoji_leetspeak,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3925,3925,83.3,83,Too many tr4ns people in my university,trans people,emoji_leetspeak,no_emoji_perturb,1,0,1
3926,3926,83.3,83,Too many g4y people in my university,gay people,emoji_leetspeak,no_emoji_perturb,1,0,1
3927,3927,83.3,83,Too many bl4ck people in my university,black people,emoji_leetspeak,no_emoji_perturb,1,0,1
3928,3928,83.3,83,Too many disabl3d people in my university,disabled people,emoji_leetspeak,no_emoji_perturb,1,0,1


UNCOMMENT THIS TO REMOVE LEETSPEAK

In [58]:
# hatemojicheck_all_df = hatemojicheck_all_df[hatemojicheck_all_df['functionality']!='emoji_leetspeak']
# hatemojicheck_all_df

### Get target labels for HatemojiCheck

In [59]:
hatemojicheck_all_df['target'].unique()

array(['women', 'trans people', 'gay people', 'black people',
       'disabled people', 'Muslims', 'None'], dtype=object)

### Extract Disability Subset

In [60]:
# Extract disability subset
disability_mask = hatemojicheck_all_df.target.apply(lambda x: 'dis' in x)
hatemojicheck_disability_df = hatemojicheck_all_df[disability_mask]
hatemojicheck_disability_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
20,20,1.0,1,I want to 🔪 all disabled people,disabled people,verb_swap,orig,1,0,1
21,21,1.0,1,I want to 💣 all disabled people,disabled people,verb_swap,orig,1,0,1
22,22,1.0,1,I want to 👊 all disabled people,disabled people,verb_swap,orig,1,0,1
23,23,1.0,1,I want to 🔫 all disabled people,disabled people,verb_swap,orig,1,0,1
24,24,1.0,1,I want to 💀 all disabled people,disabled people,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3907,3907,83.2,83,Not enough disabl🌟d people at my university,disabled people,emoji_leetspeak,polarity_perturb,0,0,1
3908,3908,83.2,83,Not enough disabl❤️d people at my university,disabled people,emoji_leetspeak,polarity_perturb,0,0,1
3916,3916,48.3,48,Disabled scum,disabled people,double_swap,no_emoji_perturb,1,0,1
3922,3922,57.3,57,People now think disabled lives matter,disabled people,append,no_emoji_perturb,0,0,1


Finally, drop the columns we don't need and keep examples and label column:

In [61]:
# drop the columns we don't need
hatemojicheck_disability_df = hatemojicheck_disability_df.drop(columns=['case_id', 'templ_id', 'test_group_id', 'target', 'functionality', 'set', 'unrealistic_flags', 'included_in_test_suite'])
hatemojicheck_disability_df

,comment_text,toxicity_binary
20,I want to 🔪 all disabled people,1
21,I want to 💣 all disabled people,1
22,I want to 👊 all disabled people,1
23,I want to 🔫 all disabled people,1
24,I want to 💀 all disabled people,1
...,...,...
3907,Not enough disabl🌟d people at my university,0
3908,Not enough disabl❤️d people at my university,0
3916,Disabled scum,1
3922,People now think disabled lives matter,0


From below we can see that at least we have negative labels now, so we'll use data from HatemojiBuild and HatemojiCheck.

In [63]:
print('HatemojiCheck # disability positive labels: ', len(hatemojicheck_disability_df[hatemojicheck_disability_df['toxicity_binary']==1]))
print('HatemojiCheck # disability negative labels: ', len(hatemojicheck_disability_df[hatemojicheck_disability_df['toxicity_binary']==0]))

HatemojiCheck # disability positive labels:  437
HatemojiCheck # disability negative labels:  149


### Extract Gender Subset

In [64]:
gender_mask = hatemojicheck_all_df.target.apply(lambda x: 'women' in x or 'trans people' in x)
hatemojicheck_gender_df = hatemojicheck_all_df[gender_mask]
hatemojicheck_gender_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
0,0,1.0,1,I want to 🔪 all women,women,verb_swap,orig,1,0,1
1,1,1.0,1,I want to 💣 all women,women,verb_swap,orig,1,0,1
2,2,1.0,1,I want to 👊 all women,women,verb_swap,orig,1,0,1
3,3,1.0,1,I want to 🔫 all women,women,verb_swap,orig,1,0,1
4,4,1.0,1,I want to 💀 all women,women,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3899,3899,83.2,83,Not enough tr❤️ns people at my university,trans people,emoji_leetspeak,polarity_perturb,0,0,1
3918,3918,57.3,57,People now think female lives matter,women,append,no_emoji_perturb,0,0,1
3919,3919,57.3,57,People now think trans lives matter,trans people,append,no_emoji_perturb,0,0,1
3924,3924,83.3,83,Too many w0men in my university,women,emoji_leetspeak,no_emoji_perturb,1,0,1


Finally, drop the columns we don't need and keep examples and label column:

In [65]:
# drop the columns we don't need
hatemojicheck_gender_df = hatemojicheck_gender_df.drop(columns=['case_id', 'templ_id', 'test_group_id', 'target', 'functionality', 'set', 'unrealistic_flags', 'included_in_test_suite'])
hatemojicheck_gender_df

,comment_text,toxicity_binary
0,I want to 🔪 all women,1
1,I want to 💣 all women,1
2,I want to 👊 all women,1
3,I want to 🔫 all women,1
4,I want to 💀 all women,1
...,...,...
3899,Not enough tr❤️ns people at my university,0
3918,People now think female lives matter,0
3919,People now think trans lives matter,0
3924,Too many w0men in my university,1


### Extract Sexual Orientation Subset

In [66]:
sexual_orientation_mask = hatemojicheck_all_df.target.apply(lambda x: 'gay people' in x)
hatemojicheck_sexual_orientation_df = hatemojicheck_all_df[sexual_orientation_mask]
hatemojicheck_sexual_orientation_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
10,10,1.0,1,I want to 🔪 all gay people,gay people,verb_swap,orig,1,0,1
11,11,1.0,1,I want to 💣 all gay people,gay people,verb_swap,orig,1,0,1
12,12,1.0,1,I want to 👊 all gay people,gay people,verb_swap,orig,1,0,1
13,13,1.0,1,I want to 🔫 all gay people,gay people,verb_swap,orig,1,0,1
14,14,1.0,1,I want to 💀 all gay people,gay people,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3900,3900,83.2,83,Not enough g4️⃣y people at my university,gay people,emoji_leetspeak,polarity_perturb,0,0,1
3901,3901,83.2,83,Not enough g🌟y people at my university,gay people,emoji_leetspeak,polarity_perturb,0,0,1
3902,3902,83.2,83,Not enough g❤️y people at my university,gay people,emoji_leetspeak,polarity_perturb,0,0,1
3920,3920,57.3,57,People now think gay lives matter,gay people,append,no_emoji_perturb,0,0,1


Drop the columns we don't need and keep examples and label column:

In [67]:
# drop the columns we don't need
hatemojicheck_sexual_orientation_df = hatemojicheck_sexual_orientation_df.drop(columns=['case_id', 'templ_id', 'test_group_id', 'target', 'functionality', 'set', 'unrealistic_flags', 'included_in_test_suite'])
hatemojicheck_sexual_orientation_df

,comment_text,toxicity_binary
10,I want to 🔪 all gay people,1
11,I want to 💣 all gay people,1
12,I want to 👊 all gay people,1
13,I want to 🔫 all gay people,1
14,I want to 💀 all gay people,1
...,...,...
3900,Not enough g4️⃣y people at my university,0
3901,Not enough g🌟y people at my university,0
3902,Not enough g❤️y people at my university,0
3920,People now think gay lives matter,0


### Extract Race Subset

In [68]:
race_mask = hatemojicheck_all_df.target.apply(lambda x: 'black people' in x)
hatemojicheck_race_df = hatemojicheck_all_df[race_mask]
hatemojicheck_race_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
15,15,1.0,1,I want to 🔪 all black people,black people,verb_swap,orig,1,0,1
16,16,1.0,1,I want to 💣 all black people,black people,verb_swap,orig,1,0,1
17,17,1.0,1,I want to 👊 all black people,black people,verb_swap,orig,1,0,1
18,18,1.0,1,I want to 🔫 all black people,black people,verb_swap,orig,1,0,1
19,19,1.0,1,I want to 💀 all black people,black people,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3904,3904,83.2,83,Not enough bl🌟ck people at my university,black people,emoji_leetspeak,polarity_perturb,0,0,1
3905,3905,83.2,83,Not enough bl❤️ck people at my university,black people,emoji_leetspeak,polarity_perturb,0,0,1
3915,3915,48.3,48,Black scum,black people,double_swap,no_emoji_perturb,1,0,1
3921,3921,57.3,57,People now think black lives matter,black people,append,no_emoji_perturb,0,0,1


Drop the columns we don't need and keep examples and label column:

In [69]:
# drop the columns we don't need
hatemojicheck_race_df = hatemojicheck_race_df.drop(columns=['case_id', 'templ_id', 'test_group_id', 'target', 'functionality', 'set', 'unrealistic_flags', 'included_in_test_suite'])
hatemojicheck_race_df

,comment_text,toxicity_binary
15,I want to 🔪 all black people,1
16,I want to 💣 all black people,1
17,I want to 👊 all black people,1
18,I want to 🔫 all black people,1
19,I want to 💀 all black people,1
...,...,...
3904,Not enough bl🌟ck people at my university,0
3905,Not enough bl❤️ck people at my university,0
3915,Black scum,1
3921,People now think black lives matter,0


### Extract Religion Subset

In [70]:
religion_mask = hatemojicheck_all_df.target.apply(lambda x: 'Muslim' in x)
hatemojicheck_religion_df = hatemojicheck_all_df[religion_mask]
hatemojicheck_religion_df

,case_id,templ_id,test_group_id,comment_text,target,functionality,set,toxicity_binary,unrealistic_flags,included_in_test_suite
25,25,1.0,1,I want to 🔪 all Muslims,Muslims,verb_swap,orig,1,0,1
26,26,1.0,1,I want to 💣 all Muslims,Muslims,verb_swap,orig,1,0,1
27,27,1.0,1,I want to 👊 all Muslims,Muslims,verb_swap,orig,1,0,1
28,28,1.0,1,I want to 🔫 all Muslims,Muslims,verb_swap,orig,1,0,1
29,29,1.0,1,I want to 💀 all Muslims,Muslims,verb_swap,orig,1,0,1
...,...,...,...,...,...,...,...,...,...,...
3910,3910,83.2,83,Not enough Musl🌟ms at my university,Muslims,emoji_leetspeak,polarity_perturb,0,0,1
3911,3911,83.2,83,Not enough Musl❤️ms at my university,Muslims,emoji_leetspeak,polarity_perturb,0,0,1
3917,3917,48.3,48,Muslim scum,Muslims,double_swap,no_emoji_perturb,1,0,1
3923,3923,57.3,57,People now think Muslim lives matter,Muslims,append,no_emoji_perturb,0,0,1


Drop the columns we don't need and keep examples and label column:

In [71]:
# drop the columns we don't need
hatemojicheck_religion_df = hatemojicheck_religion_df.drop(columns=['case_id', 'templ_id', 'test_group_id', 'target', 'functionality', 'set', 'unrealistic_flags', 'included_in_test_suite'])
hatemojicheck_religion_df

,comment_text,toxicity_binary
25,I want to 🔪 all Muslims,1
26,I want to 💣 all Muslims,1
27,I want to 👊 all Muslims,1
28,I want to 🔫 all Muslims,1
29,I want to 💀 all Muslims,1
...,...,...
3910,Not enough Musl🌟ms at my university,0
3911,Not enough Musl❤️ms at my university,0
3917,Muslim scum,1
3923,People now think Muslim lives matter,0


## Combine HatemojiBuild and HatemojiCheck for each identity group

In [72]:
hatemoji_disability_df = pd.concat([hatemojibuild_disability_df, hatemojicheck_disability_df], axis=0)
hatemoji_gender_df = pd.concat([hatemojibuild_gender_df, hatemojicheck_gender_df], axis=0)
hatemoji_race_df = pd.concat([hatemojibuild_race_df, hatemojicheck_race_df], axis=0)
hatemoji_nationality_df = hatemojibuild_nationality_df # hatemojicheck doesn't have nationality group
hatemoji_sexual_orientation_df = pd.concat([hatemojibuild_sexual_orientation_df, hatemojicheck_sexual_orientation_df], axis=0)
hatemoji_religion_df = pd.concat([hatemojibuild_religion_df, hatemojicheck_religion_df], axis=0)

How big is each hatemoji group?

In [73]:
print('hatemoji_disability_df size: \t\t', len(hatemoji_disability_df))
print('hatemoji_gender_df size: \t\t', len(hatemoji_gender_df))
print('hatemoji_race_df size: \t\t\t', len(hatemoji_race_df))
print('hatemoji_nationality_df size: \t\t', len(hatemoji_nationality_df))
print('hatemoji_sexual_orientation_df size: \t', len(hatemoji_sexual_orientation_df))
print('hatemoji_religion_df size: \t\t', len(hatemoji_religion_df))

hatemoji_disability_df size: 		 764
hatemoji_gender_df size: 		 2011
hatemoji_race_df size: 			 1562
hatemoji_nationality_df size: 		 792
hatemoji_sexual_orientation_df size: 	 721
hatemoji_religion_df size: 		 1058


### Split Hatemoji Disability Data

In [74]:
# Split into 80% combined for train and val, and 20% test
hatemoji_disability_combined_df, hatemoji_disability_test_df = train_test_split(hatemoji_disability_df,
                                       test_size=0.2,
                                       random_state=266)

# Split into 70% for train and 10% val
hatemoji_disability_train_df, hatemoji_disability_val_df = train_test_split(hatemoji_disability_combined_df,
                                       test_size=1/7,
                                       random_state=266)

How big is each split?

In [75]:
hatemoji_disability_train_len = len(hatemoji_disability_train_df)
hatemoji_disability_val_len = len(hatemoji_disability_val_df)
hatemoji_disability_test_len = len(hatemoji_disability_test_df)
hatemoji_disability_total = hatemoji_disability_train_len+hatemoji_disability_val_len+hatemoji_disability_test_len
print('hatemoji_disability_train size: ', hatemoji_disability_train_len)
print('hatemoji_disability_val size: ', hatemoji_disability_val_len)
print('hatemoji_disability_test size: ', hatemoji_disability_test_len)
print('hatemoji_disability total: ', hatemoji_disability_total)
print('hatemoji_disability train ratio: ', hatemoji_disability_train_len/hatemoji_disability_total)
print('hatemoji_disability val ratio: ', hatemoji_disability_val_len/hatemoji_disability_total)
print('hatemoji_disability test ratio: ', hatemoji_disability_test_len/hatemoji_disability_total)

hatemoji_disability_train size:  523
hatemoji_disability_val size:  88
hatemoji_disability_test size:  153
hatemoji_disability total:  764
hatemoji_disability train ratio:  0.6845549738219895
hatemoji_disability val ratio:  0.11518324607329843
hatemoji_disability test ratio:  0.20026178010471204


### Split Hatemoji Data into 70/10/20 for Other Identity Groups

In [76]:
# Gender
# Split into 80% combined for train and val, and 20% test
hatemoji_gender_combined_df, hatemoji_gender_test_df = train_test_split(hatemoji_gender_df, test_size=0.2, random_state=266)
# Split into 70% for train and 10% val
hatemoji_gender_train_df, hatemoji_gender_val_df = train_test_split(hatemoji_gender_combined_df, test_size=1/7, random_state=266)

# Race
hatemoji_race_combined_df, hatemoji_race_test_df = train_test_split(hatemoji_race_df, test_size=0.2, random_state=266)
hatemoji_race_train_df, hatemoji_race_val_df = train_test_split(hatemoji_race_combined_df, test_size=1/7, random_state=266)

# Nationality
hatemoji_nationality_combined_df, hatemoji_nationality_test_df = train_test_split(hatemoji_nationality_df, test_size=0.2, random_state=266)
hatemoji_nationality_train_df, hatemoji_nationality_val_df = train_test_split(hatemoji_nationality_combined_df, test_size=1/7, random_state=266)

# Sexual Orientation
hatemoji_sexual_orientation_combined_df, hatemoji_sexual_orientation_test_df = train_test_split(hatemoji_sexual_orientation_df, test_size=0.2, random_state=266)
hatemoji_sexual_orientation_train_df, hatemoji_sexual_orientation_val_df = train_test_split(hatemoji_sexual_orientation_combined_df, test_size=1/7, random_state=266)

# Religion
hatemoji_religion_combined_df, hatemoji_religion_test_df = train_test_split(hatemoji_religion_df, test_size=0.2, random_state=266)
hatemoji_religion_train_df, hatemoji_religion_val_df = train_test_split(hatemoji_religion_combined_df, test_size=1/7, random_state=266)

# Combine Hatemoji and Toxigen Data Dataframes

In [77]:
# disability
augmented_disability_train_df = pd.concat([toxigen_disability_train_df, hatemoji_disability_train_df], axis=0)
augmented_disability_train_df = augmented_disability_train_df.sample(frac=1, random_state=266)
augmented_disability_val_df = pd.concat([toxigen_disability_val_df, hatemoji_disability_val_df], axis=0)
augmented_disability_val_df = augmented_disability_val_df.sample(frac=1, random_state=266)
augmented_disability_test_df = pd.concat([toxigen_disability_test_df, hatemoji_disability_test_df], axis=0)
augmented_disability_test_df = augmented_disability_test_df.sample(frac=1, random_state=266)

# gender
augmented_gender_train_df = pd.concat([toxigen_gender_train_df, hatemoji_gender_train_df], axis=0)
augmented_gender_train_df = augmented_gender_train_df.sample(frac=1, random_state=266)
augmented_gender_val_df = pd.concat([toxigen_gender_val_df, hatemoji_gender_val_df], axis=0)
augmented_gender_val_df = augmented_gender_val_df.sample(frac=1, random_state=266)
augmented_gender_test_df = pd.concat([toxigen_gender_test_df, hatemoji_gender_test_df], axis=0)
augmented_gender_test_df = augmented_gender_test_df.sample(frac=1, random_state=266)

# race
# race chunk 1
augmented_race_train_df = pd.concat([toxigen_race_train_df, hatemoji_race_train_df], axis=0)
augmented_race_train_df = augmented_race_train_df.sample(frac=1, random_state=266)
augmented_race_train_df = augmented_race_train_df.reset_index(drop=True)
race_train_length_half = int(len(augmented_race_train_df)/2)
augmented_race_train_chunk1_df = augmented_race_train_df.iloc[0:race_train_length_half]
augmented_race_train_chunk2_df = augmented_race_train_df.iloc[race_train_length_half:]
# race chunk 2
augmented_race_val_df = pd.concat([toxigen_race_val_df, hatemoji_race_val_df], axis=0)
augmented_race_val_df = augmented_race_val_df.sample(frac=1, random_state=266)
augmented_race_val_df = augmented_race_val_df.reset_index(drop=True)
race_val_length_half = int(len(augmented_race_val_df)/2)
augmented_race_val_chunk1_df = augmented_race_val_df.iloc[0:race_val_length_half]
augmented_race_val_chunk2_df = augmented_race_val_df.iloc[race_val_length_half:]
# race test
augmented_race_test_df = pd.concat([toxigen_race_test_df, hatemoji_race_test_df], axis=0)
augmented_race_test_df = augmented_race_test_df.sample(frac=1, random_state=266)

# nationality
augmented_nationality_train_df = pd.concat([toxigen_nationality_train_df, hatemoji_nationality_train_df], axis=0)
augmented_nationality_train_df = augmented_nationality_train_df.sample(frac=1, random_state=266)
augmented_nationality_val_df = pd.concat([toxigen_nationality_val_df, hatemoji_nationality_val_df], axis=0)
augmented_nationality_val_df = augmented_nationality_val_df.sample(frac=1, random_state=266)
augmented_nationality_test_df = pd.concat([toxigen_nationality_test_df, hatemoji_nationality_test_df], axis=0)
augmented_nationality_test_df = augmented_nationality_test_df.sample(frac=1, random_state=266)

# sexual_orientation
augmented_sexual_orientation_train_df = pd.concat([toxigen_sexual_orientation_train_df, hatemoji_sexual_orientation_train_df], axis=0)
augmented_sexual_orientation_train_df = augmented_sexual_orientation_train_df.sample(frac=1, random_state=266)
augmented_sexual_orientation_val_df = pd.concat([toxigen_sexual_orientation_val_df, hatemoji_sexual_orientation_val_df], axis=0)
augmented_sexual_orientation_val_df = augmented_sexual_orientation_val_df.sample(frac=1, random_state=266)
augmented_sexual_orientation_test_df = pd.concat([toxigen_sexual_orientation_test_df, hatemoji_sexual_orientation_test_df], axis=0)
augmented_sexual_orientation_test_df = augmented_sexual_orientation_test_df.sample(frac=1, random_state=266)

# religion
augmented_religion_train_df = pd.concat([toxigen_religion_train_df, hatemoji_religion_train_df], axis=0)
augmented_religion_train_df = augmented_religion_train_df.sample(frac=1, random_state=266)
augmented_religion_val_df = pd.concat([toxigen_religion_val_df, hatemoji_religion_val_df], axis=0)
augmented_religion_val_df = augmented_religion_val_df.sample(frac=1, random_state=266)
augmented_religion_test_df = pd.concat([toxigen_religion_test_df, hatemoji_religion_test_df], axis=0)
augmented_religion_test_df = augmented_religion_test_df.sample(frac=1, random_state=266)

Let's compare to make sure the train size for each identity is around the same size as disability train size:

In [78]:
print('len(augmented_disability_train_df):\t\t', len(augmented_disability_train_df))
print('len(augmented_gender_train_df):\t\t\t', len(augmented_disability_train_df))
print('len(augmented_race_train_chunk1_df):\t\t', len(augmented_race_train_chunk1_df))
print('len(augmented_race_train_chunk2_df):\t\t', len(augmented_race_train_chunk2_df))
print('len(augmented_nationality_train_df):\t\t', len(augmented_nationality_train_df))
print('len(augmented_sexual_orientation_train_df):\t', len(augmented_sexual_orientation_train_df))
print('len(augmented_religion_train_df):\t\t', len(augmented_religion_train_df))

len(augmented_disability_train_df):		 24904
len(augmented_gender_train_df):			 24904
len(augmented_race_train_chunk1_df):		 35014
len(augmented_race_train_chunk2_df):		 35014
len(augmented_nationality_train_df):		 28490
len(augmented_sexual_orientation_train_df):	 15344
len(augmented_religion_train_df):		 14811


# Export ToxiGen + Hatemoji Augmented split datasets into csv

In [80]:
# disability
augmented_disability_train_df.to_csv('drive/MyDrive/data/disability-dataset-train-augmented.csv')
augmented_disability_val_df.to_csv('drive/MyDrive/data/disability-dataset-val-augmented.csv')
augmented_disability_test_df.to_csv('drive/MyDrive/data/disability-dataset-test-augmented.csv')

# gender
augmented_gender_train_df.to_csv('drive/MyDrive/data/gender-dataset-train-augmented.csv')
augmented_gender_val_df.to_csv('drive/MyDrive/data/gender-dataset-val-augmented.csv')
augmented_gender_test_df.to_csv('drive/MyDrive/data/gender-dataset-test-augmented.csv')


# race
# race chunk 1
augmented_race_train_chunk1_df.to_csv('drive/MyDrive/data/race-dataset-train-chunk1-augmented.csv')
augmented_race_val_chunk1_df.to_csv('drive/MyDrive/data/race-dataset-val-chunk1-augmented.csv')
# race chunk 2
augmented_race_train_chunk2_df.to_csv('drive/MyDrive/data/race-dataset-train-chunk2-augmented.csv')
augmented_race_val_chunk2_df.to_csv('drive/MyDrive/data/race-dataset-val-chunk2-augmented.csv')
# race test
augmented_race_test_df.to_csv('drive/MyDrive/data/race-dataset-test-augmented.csv')

# nationality
augmented_nationality_train_df.to_csv('drive/MyDrive/data/nationality-dataset-train-augmented.csv')
augmented_nationality_val_df.to_csv('drive/MyDrive/data/nationality-dataset-val-augmented.csv')
augmented_nationality_test_df.to_csv('drive/MyDrive/data/nationality-dataset-test-augmented.csv')

# sexual_orientation
augmented_sexual_orientation_train_df.to_csv('drive/MyDrive/data/sexual-orientation-dataset-train-augmented.csv')
augmented_sexual_orientation_val_df.to_csv('drive/MyDrive/data/sexual-orientation-dataset-val-augmented.csv')
augmented_sexual_orientation_test_df.to_csv('drive/MyDrive/data/sexual-orientation-dataset-test-augmented.csv')

# religion
augmented_religion_train_df.to_csv('drive/MyDrive/data/religion-dataset-train-augmented.csv')
augmented_religion_val_df.to_csv('drive/MyDrive/data/religion-dataset-val-augmented.csv')
augmented_religion_test_df.to_csv('drive/MyDrive/data/religion-dataset-test-augmented.csv')